In [21]:
import numpy as np
import cv2 as cv
import glob

In [22]:
chessboardSize = (24,17)
frameSize = (1440,1080)

In [23]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


In [24]:
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)
# size_of_chessboard_squares_mm = 20
# objp = objp * size_of_chessboard_squares_mm

In [25]:
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
#imgpointsR = [] # 2d points in image plane.


# imagesLeft = sorted(glob.glob('images/stereoLeft/*.png'))
# imagesRight = sorted(glob.glob('images/stereoRight/*.png'))

In [47]:
images=glob.glob('*.png')

In [48]:
for image in images:
    print(image)
    img=cv.imread(image)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)# convert to gray scale
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)
    # If found, add object points, image point (after refining them)
    if ret == True:

        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)

        # Draw and display the Corners
        cv. drawChessboardCorners(img, chessboardSize, corners2, ret)
        cv.imshow("img", img)
        cv.waitKey(1000)

cv.destroyAllWindows()
    

1.png
11.png
12.png
13.png
14.png
15.png
2.png
3.png
4.png
5.png
picResult2.png
picResult9.png


In [49]:
ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

print("Camera Calibration:", ret)
print("\nCamera Matrix\n", cameraMatrix)
print("\nDistortion Parameters:\n ", dist)
print("\nRotation Vectors: \n", rvecs)
print("\n Translation Vectors: \n", tvecs)

Camera Calibration: 0.7305057659342371

Camera Matrix
 [[2.97985730e+03 0.00000000e+00 7.77703374e+02]
 [0.00000000e+00 2.97354325e+03 5.23240343e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

Distortion Parameters:
  [[ -1.42475885   3.81851365   0.04040333  -0.02745213 -13.17832976]]

Rotation Vectors: 
 [array([[-0.03072345],
       [ 0.08695877],
       [-1.58677047]]), array([[-0.03072345],
       [ 0.08695877],
       [-1.58677047]]), array([[-0.01600448],
       [ 0.0728325 ],
       [-1.5950843 ]]), array([[0.04784103],
       [0.08862734],
       [0.03935033]]), array([[-0.01851699],
       [ 0.04084231],
       [ 0.02931657]]), array([[-0.03790481],
       [ 0.02197888],
       [ 0.01004116]]), array([[-0.03072345],
       [ 0.08695877],
       [-1.58677047]]), array([[-0.01600448],
       [ 0.0728325 ],
       [-1.5950843 ]]), array([[0.04784103],
       [0.08862734],
       [0.03935033]]), array([[-0.01851699],
       [ 0.04084231],
       [ 0.02931657]]), array([[-

In [51]:
img= cv.imread("11.png")
h, w = img.shape[:2]
newCameraMatrix, roi= cv.getOptimalNewCameraMatrix(cameraMatrix,dist, (w,h), 1, (w,h) )

In [52]:
#undistort
dst= cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)
#crop the image
x,y,w,h= roi
dst= dst[ y:y+h, x:x+w]
cv.imwrite("picResult9.png", dst)

True

In [53]:
#undistort with remapping
mapx, mapy= cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst= cv.remap(img,mapx,mapy,cv.INTER_LINEAR)

#crop the image
x,y,w,h =roi
dst= dst[y:y+ h, x:x+w]
cv.imwrite("picResult2.png", dst)

True

In [54]:
# Reprojection Error
mean_error = 0

for i in range(len(objpoints)):
    imgPoint2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error= cv.norm(imgpoints[i], imgPoint2, cv.NORM_L2)/len(imgPoint2)
    mean_error += error

print("\ntotal error: {}" .format(mean_error/len(objpoints)))
print("\n\n\n")


total error: 0.024603066748227785




